# 0. Setup

Load the modules for training.

In [ ]:
%matplotlib inline

import os
os.environ["CUDA_VISIBLE_DEVICES"]= "3" 

import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow 
from tensorflow.keras import layers
from keras.models import Model
from sklearn.utils import shuffle
from imgaug import augmenters as iaa
import pandas as pd
import random
from keras import backend as K

from keras.models import Sequential
import warnings
warnings.filterwarnings(action='ignore')

# 1. Laoding the dataset

- Load the preprocessed dataset
- The size of the dataset is (R, L). In our setting, we define R, L = 224, 224
- The first session contains 336 subjects with 6 fingerprint images
- The first session contains 160 subjects with 6 fingerprint images

In [ ]:
R, L = 224, 224

FIRST_SESSION_DATASET = 'Define the stored first session data path'
SECOND_SESSION_DATASET = 'Define the stored first session data path'

session1 = np.load(FIRST_SESSION_DATASET) 
session1 = session1.reshape([336, 6, R, L,1])

session2 = np.load(SECOND_SESSION_DATASET)
session2 = session2.reshape([160, 6, R, L,1])

print(session1.shape, session2.shape)

# 2. Construction of the data pairs.

## 2-1. Setting the train set
- Choose the first 136 subjects in the first session dataset. (It was drawn twice from the Genuine set to prevent the imbalance in the data ratio.)
- Choose the first 160 subjects in the second session dataset.
- DB_LIST_T_G contains pairs of genuine dataset and DB_LIST_T_I contains pairs of the imposter set.

In [ ]:
from PIL import Image
import numpy as np
import cv2

DB_LIST_T_G = []
DB_LIST_T_I = []

score = 0

for i in range(0, 136):
    for j in range(0, 5):
        for k in range(j+1, 6):
            photo1 = session1[i][j]/255.
            photo2 = session1[i][k]/255.
            DB_LIST_T_G.append([photo1, photo2, 1.0])

for i in range(0, 136):
    for j in range(0, 5):
        for k in range(j+1, 6):
            photo1 = session1[i][j]/255.
            photo2 = session1[i][k]/255.
            DB_LIST_T_G.append([photo2, photo1, 1.0])           
    
for i in range(0, 160):
    for j in range(1, 5):
        for k in range(j+1, 6):
            photo1 = session2[i][j]/255.
            photo2 = session2[i][k]/255.

            DB_LIST_T_G.append([photo1, photo2, 1.0])
            
for i in range(0, 160):
    for j in range(1, 5):
        for k in range(j+1, 6):
            photo1 = session2[i][j]/255.
            photo2 = session2[i][k]/255.
            DB_LIST_T_G.append([photo2, photo1, 1.0])
            
for i in range(0, 136):
    for j in range(i+1, 137):
        k1, k2 = random.randint(0,5), random.randint(0,5)
        photo1 = session1[i][k1]/255.
        photo2 = session1[j][k2]/255.
        DB_LIST_T_I.append([photo1, photo2, 0.0])

for i in range(0, 159):
    for j in range(i+1, 160):
        k1, k2 = random.randint(0,5), random.randint(0,5)
        photo1 = session2[i][k1]/255.
        photo2 = session2[j][k2]/255.
        DB_LIST_T_I.append([photo1, photo2, 0.0])

DB_LIST_T_G = np.array(DB_LIST_T_G)
DB_LIST_T_I = np.array(DB_LIST_T_I)
np.random.shuffle(DB_LIST_T_G) 
np.random.shuffle(DB_LIST_T_I) 

## 2-2. Setting the test set
- Choose the last 200 subjects in the first session dataset. 
- DB_LIST_TEST contains pairs of test dataset.

In [ ]:
DB_LIST_TEST = []

score = 0
for i in range(136, 336):
    for j in range(0, 5):
        for k in range(j+1, 6):
            photo1 = session1[i][j]/255.
            photo2 = session1[i][k]/255.
            DB_LIST_TEST.append([photo1, photo2, 1.0])

for i in range(136, 335):
    for j in range(i+1, 336):
        k1, k2 = random.randint(0,5), random.randint(0,5)
        photo1 = session1[i][k1]/255.
        photo2 = session1[j][k2]/255.
        DB_LIST_TEST.append([photo1, photo2, 0.0])
        
DB_LIST_TEST = np.array(DB_LIST_TEST)                
np.random.shuffle(DB_LIST_TEST)

# 3. Setting Parameters

- For augmentation

In [ ]:
BATCH_SIZE = 32
INPUT_SIZE = (R, L, 1) 
EPOCHS = 150

train_seq = iaa.Sequential([
    iaa.GaussianBlur(sigma=(0, 0.3)),
    iaa.Dropout((0.01, 0.15), per_channel=0.5),
    iaa.Affine(
        translate_percent={"x": (-0.15, 0.2), "y": (-0.15, 0.2)},
        order=[0, 1],
        cval=1
    )
], random_order=True) 

# 4. Define the Data Generator

In [ ]:
class TrainDataGenerator(tensorflow.keras.utils.Sequence):
    def __init__(self, x1, x2, batch_size=32, shuffle=True):
        self.x1 = x1
        self.x2 = x2
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.x1)/self.batch_size))

    def __getitem__(self, index):
        x1_batch, x2_batch, y_batch = [], [], []
    
        x1_list = self.x1[index*self.batch_size:(index+1)*self.batch_size]
        x2_list = self.x2[index*self.batch_size:(index+1)*self.batch_size]
        
                
        for i in range(self.batch_size):
            if random.random() > 0.5:
                a = x1_list[i]
            else:
                a = x2_list[i]
            
            x1_batch.append(a[0])
            x2_batch.append(a[1])
            y_batch.append(a[2])
             
            
            
        x1_batch, x2_batch = np.array(x1_batch).astype(np.float32), np.array(x2_batch).astype(np.float32)
        
        x1_batch = train_seq.augment_images(x1_batch)
        x2_batch = train_seq.augment_images(x2_batch)
        return [x1_batch.astype(np.float32) / 1., x2_batch.astype(np.float32) / 1.], np.array(y_batch)

    def on_epoch_end(self):
        if self.shuffle == True:
            self.x1 = shuffle(self.x1) 
            self.x2 = shuffle(self.x2) 
            
            
class TestDataGenerator(tensorflow.keras.utils.Sequence):
    def __init__(self, x, batch_size=32, shuffle=True):
        self.x = x
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        

    def __len__(self):
        return int(np.floor(len(self.x) / self.batch_size))

    def __getitem__(self, index):
        x1_batch, x2_batch, y_batch = [], [], []
        
        x_list = self.x[index*self.batch_size:(index+1)*self.batch_size]
        
        for a in x_list:
            x1_batch.append(a[0])
            x2_batch.append(a[1])
            y_batch.append(a[2])
            
        x1_batch, x2_batch = np.array(x1_batch), np.array(x2_batch)
        
        return [x1_batch.astype(np.float32) / 1., x2_batch.astype(np.float32) / 1.], np.array(y_batch)

    def on_epoch_end(self):
        if self.shuffle == True:
            self.x = shuffle(self.x)
    
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


swish = tensorflow.keras.activations.swish

def square(x):
    return x ** 2

train_gen = TrainDataGenerator(DB_LIST_T_G, DB_LIST_T_I)

# 5. Define the model

- Define the model for training

In [ ]:
x1 = layers.Input(shape=(R, L, 1))
x2 = layers.Input(shape=(R, L, 1))

inputs = layers.Input(shape=(R, L, 1))

padding = 'same'
activation = swish
pool_size = 2

feature = layers.Conv2D(32, kernel_size=3, padding=padding, strides=1)(inputs)
feature = layers.BatchNormalization()(feature) 
feature = activation(feature)
layers.Dropout(.4, input_shape=(2,))
feature = layers.MaxPooling2D(pool_size=pool_size)(feature)

feature = layers.Conv2D(64, kernel_size=3, padding=padding, strides=1)(feature)
feature = layers.BatchNormalization()(feature) 
feature = activation(feature)
layers.Dropout(.4, input_shape=(2,))
feature = layers.MaxPooling2D(pool_size=pool_size)(feature)

feature = layers.Conv2D(128, kernel_size=3, padding=padding, strides=1)(feature)
feature = layers.BatchNormalization()(feature) 
feature = activation(feature)
layers.Dropout(.4, input_shape=(2,))
feature = layers.MaxPooling2D(pool_size=pool_size)(feature)

feature = layers.Conv2D(256, kernel_size=3, padding=padding, strides=1)(feature)
feature = layers.BatchNormalization()(feature) 
feature = activation(feature)
layers.Dropout(.4, input_shape=(2,))
feature = layers.MaxPooling2D(pool_size=pool_size)(feature)

feature = layers.Conv2D(512, kernel_size=3, padding=padding, strides=1)(feature) 
feature = layers.BatchNormalization()(feature) 
feature = activation(feature)
layers.Dropout(.4, input_shape=(2,))
feature = layers.MaxPooling2D(pool_size=pool_size)(feature)

feature_model = Model(inputs=inputs, outputs=feature)

x1_net = feature_model(x1)
x2_net = feature_model(x2)

x1_net = layers.Flatten()(x1_net)
x2_net = layers.Flatten()(x2_net)
x1_net = keras.layers.UnitNormalization()(x1_net)
x2_net = keras.layers.UnitNormalization()(x2_net)

net = layers.Subtract()([x1_net, x2_net])
net = layers.Dense(16)(net)
net = square(net)
net = layers.Dense(1, activation='sigmoid')(net)

model = Model(inputs=[x1, x2], outputs=net)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', f1_m, precision_m, recall_m])

model.summary()

# 6. Train the model

- Train the defined model

In [ ]:
history = model.fit(train_gen, epochs=150, batch_size=32)

# 7. Save the model
- Save the weights of the trained feature model and model.

In [ ]:
FEATURE_MODEL_PATH = 'Define the path to store the weigths of the feature model'
MODEL_PATH = 'Define the path to store the weigths of the model'

feature_model.save(FEATURE_MODEL_PATH)
model.save(MODEL_PATH)